# Prac 6
### 18BCE259

##### Processing Text with RNN

In [1]:
!nvidia-smi

Mon Mar  8 08:50:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
logging.getLogger("tensorflow").addHandler(logging.NullHandler(logging.ERROR))

In [4]:
data = " It is well known that medical diagnosis takes more time than deciding treatment for the condition .\n It might require plenty of time and skills of medical practitioners .\n In recent times, COVID-19 spread across the globe made computer researchers work toward faster and more accurate medical diagnosis solutions for hospitals .\n This project contains the code for diagnosing pneumonia from the chest X-ray images ."
text = data.split('\n')

In [5]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='!"#$*&()*+,-/:;<=>?@[\]^_{}~')
tokenizer.fit_on_texts(text)

In [6]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

53

In [7]:
sequences = tokenizer.texts_to_sequences(text)
l = len(sequences)

In [8]:
max_len = max([len(seq) for seq in sequences])
max_len

24

In [9]:
X = []
y = sequences
for sq in sequences:
    X.append(sq[:-1])

In [10]:
max_len = max([len(seq) for seq in sequences])

In [11]:
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=max_len+1)
y = tf.keras.preprocessing.sequence.pad_sequences(y, maxlen=max_len+1)
y = tf.keras.utils.to_categorical(y, num_classes=vocab_size)

In [12]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=10),
                                    tf.keras.layers.SimpleRNN(50, return_sequences=True),
                                    tf.keras.layers.Dense(vocab_size, activation="softmax")
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 10)          530       
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, None, 50)          3050      
_________________________________________________________________
dense (Dense)                (None, None, 53)          2703      
Total params: 6,283
Trainable params: 6,283
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(optimizer="RMSprop", loss="categorical_crossentropy", metrics=['accuracy'])
model.fit(X,y, epochs=200)

Epoch 1/200
1/1 [==============================] - 1s 1s/step - loss: 3.9763 - accuracy: 0.0100
Epoch 2/200
1/1 [==============================] - 0s 25ms/step - loss: 3.9193 - accuracy: 0.0400
Epoch 3/200
1/1 [==============================] - 0s 26ms/step - loss: 3.8636 - accuracy: 0.0400
Epoch 4/200
1/1 [==============================] - 0s 27ms/step - loss: 3.8258 - accuracy: 0.0500
Epoch 5/200
1/1 [==============================] - 0s 25ms/step - loss: 3.7934 - accuracy: 0.1700
Epoch 6/200
1/1 [==============================] - 0s 28ms/step - loss: 3.7614 - accuracy: 0.3300
Epoch 7/200
1/1 [==============================] - 0s 24ms/step - loss: 3.7308 - accuracy: 0.3100
Epoch 8/200
1/1 [==============================] - 0s 27ms/step - loss: 3.6944 - accuracy: 0.3300
Epoch 9/200
1/1 [==============================] - 0s 31ms/step - loss: 3.6608 - accuracy: 0.3500
Epoch 10/200
1/1 [==============================] - 0s 31ms/step - loss: 3.6176 - accuracy: 0.3600
Epoch 11/200
1/1 [===

In [14]:
def prob_of_input_sentence(model, tokenizer, sentence):
    print ("Input Sentence:", sentence)
    encoded=tokenizer.texts_to_sequences([sentence])[0]
    encoded.insert(0, 0)
    encoded=np.array(encoded)
    encoded=np.reshape(encoded, newshape=(1, -1))
    prob = model.predict(encoded, verbose=0)
    probability=1
    for i in range (prob. shape[1]-1):
        probability = probability * prob[0, i, encoded[0, i+1]]
    print("Probability: ", probability)

In [15]:
prob_of_input_sentence(model, tokenizer, "It is well known that")

Input Sentence: It is well known that
Probability:  1.1526633691371497e-06


In [16]:
prob_of_input_sentence(model, tokenizer, "medical diagnosis takes more time than deciding")

Input Sentence: medical diagnosis takes more time than deciding
Probability:  1.26546502015892e-18


In [17]:
prob_of_input_sentence(model, tokenizer, "It is well known that medical diagnosis takes more time than deciding treatment for the condition")

Input Sentence: It is well known that medical diagnosis takes more time than deciding treatment for the condition
Probability:  8.186663421780185e-22


In [18]:
prob_of_input_sentence(model, tokenizer, "It might require plenty of time")

Input Sentence: It might require plenty of time
Probability:  2.968899546582824e-10


#### Predicting Next Chars

In [19]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()


In [20]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [21]:
tokenizer.texts_to_sequences(['First'])

[[20, 6, 9, 8, 3]]

In [22]:
tokenizer.sequences_to_texts([[20,6,9,8,3]])

['f i r s t']

In [23]:
max_id = len(tokenizer.word_index)
max_id

39

In [24]:
dataset_size = tokenizer.document_count

In [25]:
dataset_size

1115394

In [26]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

In [27]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [28]:
n_steps = 100
window_length = n_steps + 1

In [29]:
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [30]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [31]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [32]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [33]:
dataset = dataset.prefetch(1)

In [34]:
import os

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [35]:
class NvidiaUtilizationCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        text = !nvidia-smi
        text = text[9][60:65] + ' GPU utilization'
        print(text)

In [36]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation='softmax'))    
])

model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam')
steps_per_epoch = train_size // batch_size // n_steps

history = model.fit(dataset, epochs = 40, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback, NvidiaUtilizationCallback()])

Epoch 1/40
  0%  GPU utilization
313/313 [==============================] - 12s 25ms/step - loss: 2.9229
Epoch 2/40
 35%  GPU utilization
313/313 [==============================] - 8s 25ms/step - loss: 2.1208
Epoch 3/40
 57%  GPU utilization
313/313 [==============================] - 8s 24ms/step - loss: 1.8792
Epoch 4/40
 59%  GPU utilization
313/313 [==============================] - 8s 24ms/step - loss: 1.7314
Epoch 5/40
 39%  GPU utilization
313/313 [==============================] - 8s 24ms/step - loss: 1.5716
Epoch 6/40
 55%  GPU utilization
313/313 [==============================] - 8s 24ms/step - loss: 1.4022
Epoch 7/40
 60%  GPU utilization
313/313 [==============================] - 8s 24ms/step - loss: 1.3303
Epoch 8/40
 48%  GPU utilization
313/313 [==============================] - 8s 24ms/step - loss: 1.2852
Epoch 9/40
 32%  GPU utilization
313/313 [==============================] - 8s 25ms/step - loss: 1.2905
Epoch 10/40
 56%  GPU utilization
313/313 [====================

In [37]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [38]:
X_new = preprocess(["How are yo"])

In [39]:
Y_pred = np.argmax(model.predict(X_new), axis=-1)

In [40]:
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]

'u'

In [41]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [42]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [43]:
text_1 = complete_text("t", temperature=0.2)
text_1

"t the king surjoy:\nlet's that he is a royal presenc"

In [44]:
text_2 = complete_text("How are yo", temperature=1)
text_2

"How are you so honore thou of thear strue, his head's londs."

In [45]:
text_3 = complete_text("How you doin", temperature=1)
text_3

"How you doinmy duey.\n\nking richard,\nthat h'pray of not to-drea"